# 1er TP de sistemas inteligentes

Nombre: Santiago Vietto - 1802890

Este trabajo práctico se aprueba completando más del 70% del mismo. Una vez completado el notebook y reenvielo al email 2329034@ucc.edu.ar. Importante: Asegurese que su correo haya llegado.

### Entender el problema

Este conjunto de datos describe los hongos en términos de sus características físicas. Se clasifican como venenosos o comestibles.

Este conjunto de datos incluye descripciones de muestras hipotéticas correspondientes a 23 especies de hongos de láminas en las familias Agaricus y Lepiota. Cada especie se identifica como definitivamente comestible, definitivamente venenosa o de comestibilidad desconocida y no recomendada. Esta última clase se combinó con la venenosa. La guía establece claramente que no hay una regla simple para determinar la comestibilidad de un hongo. Con esa informacion:

Siendo la clase positiva la indicación que el hongo es venenoso ¿Qué le estaría pidiendo a mi clasificador qué tenga en cuenta si le pido un puntaje de precision alto?

url = 'https://gitlab.com/francisco.arduh/datasets/-/raw/main/mushroom.csv'

<10%>
Respuesta: En resumen, si se pide un puntaje de precisión alto para la clasificación de hongos venenosos, lo que se le pide al clasificador es que basicamente detecte correctamente los hongos venenosos. En donde el clasificador debe tener una alta sensibilidad y especificidad para la clase positiva, que son los hongos venenoso, pudiendo identificarlos correctamente y a su vez que minimice el número de hongos comestibles que fueron clasificados por error como venenosos.

### Obtención de datos
Importe las librerías a utilizar:

In [31]:
import pandas as pd
import os
import tarfile
import urllib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_predict, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, precision_score, f1_score
from sklearn.compose import ColumnTransformer

Respuesta: Las siguientes librerias a importar son aquellas que nos ayudaran a correr las funciones que seran utilizadas a lo largo del proyecto:


* Las 6 primeras nos serviran para proporcionar estructuras de datos y herramientas para la manipulacion y analisis de datos, interactuar con el sistema operativo, como crear directorios, obtener info sobre el sistema, trabajar con archivos de formato tar, y por ej. comprimir y descomprimir archivos, descarga de archivos de internet, vizualizar de datos en 2D en forma de graficos, histogramas, etc.
* Las demas importan diversas herramientas de la biblioteca scikit-learn (sklearn) para procesamiento de datos y modelado de ML, como por ejemplo para dividir un conjunto de datos en conjuntos de entrenamiento y prueba, de manera aleatoria, hacer validacion cruzada para evaluar el rendimiento, realizar una busqueda exhaustiva de los mejores hiperparametros, transformar variables categoricas en variables numericas, calcular diferentes metricas de evaluacion, etc.

Importe los datos:

In [2]:
# Descargamos los datos de la siguiente URL

MUSHROOM_PATH= "https://gitlab.com/francisco.arduh/datasets/-/raw/main/mushroom.csv"

# Cargamos los datos

def load_mushroom_data(mushroom_path = MUSHROOM_PATH):
    csv_path = os.path.join(mushroom_path)
    return pd.read_csv(csv_path)

# Vizualizamos los datos

mushroom = load_mushroom_data()
mushroom.describe()


,cap-shape,cap-surface,cap-color,bruises%3F,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,...,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,6,4,10,2,9,2,2,2,12,2,...,9,9,1,4,3,5,9,6,7,2
top,b'x',b'y',b'n',b'f',b'n',b'f',b'c',b'b',b'b',b't',...,b'w',b'w',b'p',b'w',b'o',b'p',b'w',b'v',b'd',b'e'
freq,3656,3244,2284,4748,3528,7914,6812,5612,1728,4608,...,4464,4384,8124,7924,7488,3968,2388,4040,3148,4208


Respuesta: Entonces, como primera medida desacrgamos los datos. Podemos ver que la URL ya contiene el archivo CSV, por lo que simplemente debemos cargar los datos, y no crear el archivo comprimido para luego descomprimirlo en el CSV. Creamos la ruta o path del archivo, y por ultimo vizualizamos de forma ordenada el conjunto de datos que posee.

<10%> Realice una exploración de los datos, compruebe si existe faltante de datos en alguna columna, que tipo de distribución tienen los datos, si alguna columna tiene muchos datos únicos.

In [3]:
# Obtenemos informacion del conjunto de datos, mediante la funcion info()

print(mushroom.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   cap-shape                 8124 non-null   object
 1   cap-surface               8124 non-null   object
 2   cap-color                 8124 non-null   object
 3   bruises%3F                8124 non-null   object
 4   odor                      8124 non-null   object
 5   gill-attachment           8124 non-null   object
 6   gill-spacing              8124 non-null   object
 7   gill-size                 8124 non-null   object
 8   gill-color                8124 non-null   object
 9   stalk-shape               8124 non-null   object
 10  stalk-root                8124 non-null   object
 11  stalk-surface-above-ring  8124 non-null   object
 12  stalk-surface-below-ring  8124 non-null   object
 13  stalk-color-above-ring    8124 non-null   object
 14  stalk-color-below-ring  

In [4]:
# Comprobamos si hay de valores nulos o faltantes en alguna columna

print(mushroom.isnull().sum())

cap-shape                   0
cap-surface                 0
cap-color                   0
bruises%3F                  0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
class                       0
dtype: int64


In [5]:
# Vemos la cantidad de valores unicos en cada columna

for column in mushroom.columns:
    print(column, mushroom[column].nunique())

cap-shape 6
cap-surface 4
cap-color 10
bruises%3F 2
odor 9
gill-attachment 2
gill-spacing 2
gill-size 2
gill-color 12
stalk-shape 2
stalk-root 5
stalk-surface-above-ring 4
stalk-surface-below-ring 4
stalk-color-above-ring 9
stalk-color-below-ring 9
veil-type 1
veil-color 4
ring-number 3
ring-type 5
spore-print-color 9
population 6
habitat 7
class 2


Respuesta: Una vez obtenido los resultados en estas 3 ejecuciones, podemos observar que:
* Todas las columnas del conjunto de datos son objetos o variables categoricas, estos son diferentes a los atributos numericos, ya que son variables que pueden 
tomar valores continuos o discretos, similar a texto o string.
* Comprobamos que no hay valores faltantes en ninguna columna.
* Comprobamos que todas las columnas tienen mas de un valor unico.

<5%> Separar X_train, y_train, X_test, y_test

In [6]:
# Dividimos los datos segun train y test: 

X = mushroom.drop('class', axis=1)     # Todas las columnas excepto la clase son características
y = mushroom['class']                  # La columna "class" es la etiqueta

y = pd.Categorical(mushroom['class']).codes.astype(np.uint8)  # Transformamos la columna "class" en numerica
print(y)

[1 0 0 ... 0 1 0]


In [7]:
# Separamos el conjuntos de entrenamiento (80%) y prueba (20%)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Vemos las variables

print("Tamaño de X_train:", X_train.shape)
print("Tamaño de X_test:", X_test.shape)
print("Tamaño de y_train:", y_train.shape)
print("Tamaño de y_test:", y_test.shape)

Tamaño de X_train: (6499, 22)
Tamaño de X_test: (1625, 22)
Tamaño de y_train: (6499,)
Tamaño de y_test: (1625,)


Respuesta: Analizando estas lineas tenemos que:

* Primero preparamos los datos para un analisis de clasificacion de hongos comestibles o venenosos. Vemos que en X guardamos todas las columnas, menos la columna "class", porque X es una matriz de características que se va a usar para predecir la etiqueta "class", y esta ultima se almacena en la variable "y" que es una serie de etiquetas "class" correspondientes a cada instancia de la matriz X. Ademas, se convertimos la columna "class" de los datos de hongos a numeros enteros para facilitar su manejo y asi no manipular strings.
* Luego lo que hacemos es dividir el conjunto de datos en un conjunto de entrenamiento y otro de prueba, en donde X es el conjunto de características o variables explicativas, y es el conjunto de etiquetas o variable objetivo, indicamos que el conjunto de prueba tendra el 20% de los datos y el conjunto de entrenamiento tendra el 80% restante, y random_state=42 es una semilla que se utiliza para asegurar que la division sea reproducible.
* Por ultimo visualizamos las variables creadas.

### Procesado y transformación de datos
<10%> ¿Qué transformaciones debería aplicar? Especifique y justifique por grupo de columnas.

Por ejemplo: columna A, B y C aplicaría transformación X porque ...

Escriba los nombres de las columna en una lista por cada grupo:

Respuesta: Dado a que todas las columnas del conjunto de datos mushroom son de tipo categoricas, se pueden aplicar las siguientes transformaciones:

* Columnas categoricas: en todas estas columnas deberiamos aplicar una codificacion "one-hot", ya que todas estas columnas tienen valores discretos. Las columnas son, 'cap-shape', 'cap-surface', 'cap-color', 'bruises%3F', 'odor', 'gill-attachment, 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat'.

<15%> A continuación escriba el código para realizar las transformaciones transformaciones:

In [9]:
# Separar variables numericas y categoricas

mushroom_cat = X_train[['cap-shape', 'cap-surface', 'cap-color', 'bruises%3F', 'odor', 'gill-attachment', 
                'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 
                'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 
                'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 
                'spore-print-color', 'population', 'habitat']]

cat_encoder = OneHotEncoder()
mushroom_cat_X_train_1hot = cat_encoder.fit_transform(mushroom_cat)
mushroom_cat_X_test_1hot = cat_encoder.transform(X_test)
mushroom_cat_X_train_1hot

<6499x117 sparse matrix of type '<class 'numpy.float64'>'
	with 142978 stored elements in Compressed Sparse Row format>

In [10]:
# Convertimos la matriz dispersa en densa

mushroom_cat_X_train_1hot.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
# Obtenemos la lista de categorías

cat_encoder.categories_

[array(["b'b'", "b'c'", "b'f'", "b'k'", "b's'", "b'x'"], dtype=object),
 array(["b'f'", "b'g'", "b's'", "b'y'"], dtype=object),
 array(["b'b'", "b'c'", "b'e'", "b'g'", "b'n'", "b'p'", "b'r'", "b'u'",
        "b'w'", "b'y'"], dtype=object),
 array(["b'f'", "b't'"], dtype=object),
 array(["b'a'", "b'c'", "b'f'", "b'l'", "b'm'", "b'n'", "b'p'", "b's'",
        "b'y'"], dtype=object),
 array(["b'a'", "b'f'"], dtype=object),
 array(["b'c'", "b'w'"], dtype=object),
 array(["b'b'", "b'n'"], dtype=object),
 array(["b'b'", "b'e'", "b'g'", "b'h'", "b'k'", "b'n'", "b'o'", "b'p'",
        "b'r'", "b'u'", "b'w'", "b'y'"], dtype=object),
 array(["b'e'", "b't'"], dtype=object),
 array(["b'?'", "b'b'", "b'c'", "b'e'", "b'r'"], dtype=object),
 array(["b'f'", "b'k'", "b's'", "b'y'"], dtype=object),
 array(["b'f'", "b'k'", "b's'", "b'y'"], dtype=object),
 array(["b'b'", "b'c'", "b'e'", "b'g'", "b'n'", "b'o'", "b'p'", "b'w'",
        "b'y'"], dtype=object),
 array(["b'b'", "b'c'", "b'e'", "b'g'", "b'n'", 

Respuesta: Analizamos las lineas de codigo anteriores y comentamos que:

* Primero convertimos las variables categoricas que estan en el arreglo de X_tarin, en vectores one-hot. Con esta transofrmacion se crea una nueva columna para cada categoria posible y se le asigna un valor binario de 1 o 0 para indicar si esa categoria esta presente o no en la observacion. Entonces, estas nuevas columnas se llaman "atributos ficticios" porque no representan una medida real de la variable original, sino que son simplemente una forma de representar categoricamente las observaciones en un formato que puede ser entendido por los algoritmos de ML. Pero en primer lugar, luego de aplicar la codificadion, el resultado que obtenemos es una matriz dispersa con miles de columnas que representa los vectores one-hot.
* En terminos de memoria esta matriz es ineficiente, ya que se utiliza una matriz dispersa que solo almacena la ubicacion de los elementos no nulos. Entonces lo que hacemos es convertir esta matriz dispersa en una matriz NumPy densa para poder manipularla como una matriz normal.
* Por ultimo retornamos las categorias de las variables categoricas depsues de 
haber sido transformadas por el codificador One-Hot y almacenadas en el objeto cat_encoder.

### Selección de modelo y entrenamiento
 


In [12]:
# Entrenamos el modelo con regresion logistica

from sklearn.linear_model import LogisticRegression

cla_log = LogisticRegression()
cla_log.fit(mushroom_cat_X_train_1hot, y_train)

LogisticRegression()

In [13]:
# Entrenamos el modelo con RandomForestClassifier

from sklearn.ensemble import RandomForestClassifier

cla_rf = RandomForestClassifier()
cla_rf.fit(mushroom_cat_X_train_1hot, y_train)

RandomForestClassifier()

Respuesta: Dado los resultados anteriores, lo que hacemos en estas lineas es:

* Aplicamos un modelo de regresion logistica en el conjunto de datos de entrenamiento. Usamos el conjunto de datos de caracteristicas de entrenamiento (mushroom_cat_X_train_1hot) y el conjunto de datos de etiquetas de entrenamiento (y_train). Y entrenamos el modelo con estos datos proporcionados.
* Luego aplicamos un modelo RandomForestClassifier en el conjunto de datos de entrenamiento. Usamos el conjunto de datos de caracteristicas de entrenamiento (mushroom_cat_X_train_1hot) y el conjunto de datos de etiquetas de entrenamiento (y_train). Y entrenamos el modelo con estos datos proporcionados.

<15%>Utilice validación cruzada para evaluar los modelos que se importaron. Utilice el de area bajo la curva roc mas alto.
Utilizar el metodo "cross_val_predict", tenga en cuenta que la variable, "method" para el LogisticRegression es "decision_function" y para RandomForestClassifier es "predict_proba" (ver clase o libro)

In [14]:
# Aplicamos validacion cruzada para Logistic Regression

# Realizamos la validacion cruzada con 3 particiones
y_df_log = cross_val_predict(cla_log, mushroom_cat_X_train_1hot, y_train, cv=3, method='decision_function')

# Calculamos el area bajo la curva ROC
auc_roc = roc_auc_score(y_train, y_df_log)

# Vemos el resultado
print("Logistic Regression AUC:", auc_roc)

Logistic Regression AUC: 1.0


In [15]:
# Aplicamos validacion cruzada para Random Forest Classifier

# Realizamos la validacion cruzada con 3 particiones
y_probas_rf = cross_val_predict(cla_rf, mushroom_cat_X_train_1hot, y_train, cv=3, method='predict_proba')

# Calculamos el area bajo la curva ROC
aur_roc = roc_auc_score(y_train, y_probas_rf[:,1])

# Vemos el resultado
print("Random Forest Classifier AUC:", aur_roc)

Random Forest Classifier AUC: 1.0


Respuesta: En terminos generales usamos la función cross_val_predict para realizar una validacion cruzada con 3 particiones y obtener las decisiones de los dos clasificadores para cada particion de prueba. 

* Para Logistic Regression las decisiones son retornadas por el método decision_function(), luego se calcula el area bajo la curva ROC utilizando la funcion roc_auc_score, y por ultimo vemos el resultado. 
* Para RandomForest, el metodo predict_proba() indica que se deben obtener las probabilidades de las predicciones en lugar de las predicciones reales, luego se calcula el area bajo la curva ROC utilizando la funcion roc_auc_score, el valor de y_probas_rf[:,1] es un array que contiene las probabilidades predichas para la clase positiva, por lo que se selecciona la segunda columna del array para el calculo del area bajo la curva ROC, y por ultimo vemos el resultado. 


<10%>¿Por qué lo eligiria el modelo por el area bajo la curva roc?


Respuesta: El area bajo la curva ROC es una metmetrica que se utiliza para evaluar la calidad de un modelo de clasificacion binaria. Cuanto mas cercano a 1 sea el valor del area, mejor sera el modelo en terminos de su capacidad para distinguir correctamente entre las clases.

Podemos observar que en ambos modelos el resultado del area es 1.0, lo que indica que ambos modelos tienen una capacidad perfecta para distinguir entre las clases positivas y negativas. En este caso seria indistinto seleccionar cualquiera de los dos, pero destacamos situaciones en las que convendria usar uno u otro:

* La Regresion logistica es utilizada para predecir una variable categorica binaria (dos clases), a partir de un conjunto de variables predictoras. Es un modelo simple y rapido, que funciona bien cuando se trabaja con un conjunto de datos pequeño, y cuando la relacion entre las variables predictoras y la variable de respuesta es lineal o logica.

* Random Forest es un modelo que puede utilizarse tanto para clasificacion como para regresion. Es un modelo mas complejo y potente que la Regresion logistica, capaz de manejar un gran numero de características (variables predictoras) y de detectar interacciones no lineales entre ellas. Funciona mejor cuando hay una gran cantidad de datos disponibles y cuando las relaciones entre las variables son mas complejas. Por lo que considero que para este caso de analisis, este metodo es el indicado.

<15>Utilice el metodo grid search sobre el modelo elegido:

In [16]:
# Aplicamos el metodo Grid Search en el modelo de Regresion logistica

# Creamos un diccionario de hiperparametros para probar
param_grid = {'C': [0.1, 1, 10]}

# Creamos un objeto GridSearchCV
grid_log = GridSearchCV(cla_log, param_grid, cv=3)

# Ajustamos el modelo con la busqueda de cuadricula
grid_log.fit(mushroom_cat_X_train_1hot, y_train)

# Vemos los resultados
print("Mejores hiperparametros: ", grid_log.best_params_)
print("Mejor puntaje: ", grid_log.best_score_)

Mejores hiperparametros:  {'C': 1}
Mejor puntaje:  1.0


In [17]:
# Aplicamos el metodo Grid Search en el modelo RandomForest

# Creamos un diccionario de hiperparametros para probar
param_grid = { 'n_estimators': [30, 100], 'max_features': [2, 4, 6, 8] }

# Creamos un objeto GridSearchCV
grid_rf = GridSearchCV(cla_rf, param_grid, cv=3, scoring='roc_auc')

# Ajustamos el modelo con la busqueda de cuadricula
grid_rf.fit(mushroom_cat_X_train_1hot, y_train)

# Vemos los resultados
print('Mejores hiperparametros:', grid_rf.best_params_)
print('Mejor puntaje:', grid_rf.best_score_)

Mejores hiperparametros: {'max_features': 2, 'n_estimators': 30}
Mejor puntaje: 1.0


Respuesta: Aplicamos el metodo Grid Search en ambos modelos para tener una referencia. A continuacion se explica un poco el procedimiento:

* Para el caso de la Regresion logistica, C es el parametro de regularizacion de la Regresion logistica. Despues de definir y ajustar el objeto GridSearchCV, el modelo se ajusta con la búsqueda de cuadricula mediante la llamada al metodo fit() con los datos de entrenamiento mushroom_cat_X_train_1hot y y_train. Y por ultimo, se imprimen los mejores hiperparametros encontrados por la busqueda de cuadricula y el puntaje de la validación cruzada en el conjunto de entrenamiento utilizando los mejores hiperparametros.
* Para el caso de RandomForest, se define un diccionario que contiene los valores que se quieren probar para cada hiperparametro, el numero de arboles a considerar (n_estimators) y el numero maximo de variables a considerar en cada arbol (max_features). Despues de definir y ajustar el objeto GridSearchCV, el modelo se ajusta con el metodo fit, y realiza una busqueda exhaustiva de todas las combinaciones de hiperparametros y devuelve la combinación con mejor desempeño en la metrica de evaluación especificada. Entonces vemos como resultado los mejores hiperparametros encontrados y el mejor puntaje obtenido en la evaluacion cruzada.

¿Cuál es el sentido de hacer esto?

Respuesta: Tenemos que saber que Grid Search es una tecnica utilizada en ML para buscar los mejores valores de hiperparametros de un modelo, que resultan en el mejor rendimiento de prediccion, con el objetivo de maximizar su rendimiento. Los hiperparametros son parametros del modelo que no se aprenden automaticamente durante el entrenamiento y que deben ser establecidos por el usuario antes de entrenar el modelo. Algunos ejemplos de hiperparametros son la tasa de aprendizaje, la profundidad maxima del arbol de decisiones, el numero de estimadores en un modelo de bosques aleatorios, etc.

Es importante seleccionar cuidadosamente los hiperparametros y valores a incluir en la cuadricula, y se pueden utilizar tacnicas como la busqueda aleatoria o la búsqueda exhaustiva para hacer el proceso mas eficiente.

<5%>Imprima el valor de accuracy, recall, precision, f1 y auc_roc, en las regiones de validación

In [34]:
# Modelo de Regresion Logistica

# Realizamos la validacion cruzada con 3 particiones
y_pred_log = cross_val_predict(cla_log, mushroom_cat_X_train_1hot, y_train, cv=3)

# Calculamos las metricas de evaluacion
accuracy = accuracy_score(y_train, y_pred_log)
recall = recall_score(y_train, y_pred_log)
precision = precision_score(y_train, y_pred_log)
f1 = f1_score(y_train, y_pred_log)
auc_roc = roc_auc_score(y_train, y_df_log)

# Vemos los resultados
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)
print("AUC-ROC:", auc_roc)

Accuracy: 1.0
Recall: 1.0
Precision: 1.0
F1 Score: 1.0
AUC-ROC: 1.0


In [35]:
# Modelo RandomForest

# Realizamos la validacion cruzada con 3 particiones
y_pred_rf = cross_val_predict(cla_rf, mushroom_cat_X_train_1hot, y_train, cv=3)

# Calculamos las metricas de evaluacion
accuracy = accuracy_score(y_train, y_pred_rf)
recall = recall_score(y_train, y_pred_rf)
precision = precision_score(y_train, y_pred_rf)
f1 = f1_score(y_train, y_pred_rf)
aur_roc = roc_auc_score(y_train, y_probas_rf[:,1])

# Vemos los resultados
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)
print("AUC-ROC:", auc_roc)

Accuracy: 1.0
Recall: 1.0
Precision: 1.0
F1 Score: 1.0
AUC-ROC: 1.0


Respuesta: Podemos observar que las metricas dan como resultado 1.0, y esto significa que ambos modelos estan prediciendo perfectamente los resultados en esta region. Esto puede darse ya que el modelo se ajusto demasiado bien a los datos de entrenamiento y por lo tanto, no es capaz de generalizar a datos nuevos.



<5%>Imprima el valor de accuracy, recall, precision, f1 y auc_roc, en las regiones de test, 
¿Los números son parecido a lo obtenido en las regiones de validación? ¿A qué puede deberse?

In [39]:
# Modelo de Regresion Logistica

# Realizamos la validacion cruzada con 3 particiones
y_pred_log = cross_val_predict(cla_log, mushroom_cat_X_test_1hot, y_test, cv=3)

# Calculamos las metricas de evaluacion
accuracy = accuracy_score(y_test, y_pred_log)
recall = recall_score(y_test, y_pred_log)
precision = precision_score(y_test, y_pred_log)
f1 = f1_score(y_test, y_pred_log)
auc_roc = roc_auc_score(y_test, grid_log.decision_function(mushroom_cat_X_test_1hot))

# Vemos los resultados
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)
print("AUC-ROC:", auc_roc)

Accuracy: 0.9987692307692307
Recall: 0.9974424552429667
Precision: 1.0
F1 Score: 0.998719590268886
AUC-ROC: 1.0


In [38]:
# Modelo RandomForest

# Realizamos la validacion cruzada con 3 particiones
y_pred_rf = cross_val_predict(cla_rf, mushroom_cat_X_test_1hot, y_test, cv=3)

# Calculamos las metricas de evaluacion
accuracy = accuracy_score(y_test, y_pred_rf)
recall = recall_score(y_test, y_pred_rf)
precision = precision_score(y_test, y_pred_rf)
f1 = f1_score(y_test, y_pred_rf)
aur_roc = roc_auc_score(y_test, grid_rf.predict_proba(mushroom_cat_X_test_1hot)[:,1])

# Vemos los resultados
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)
print("AUC-ROC:", auc_roc)

Accuracy: 0.9987692307692307
Recall: 0.9974424552429667
Precision: 1.0
F1 Score: 0.998719590268886
AUC-ROC: 1.0


Respuesta: Los numeros obtenidos en las regiones de test son muy similares a los obtenidos en las regiones de validacion, y esto significa que ambos modelos no han sufrido de sobreajuste (overfitting) durante el entrenamiento, es decir, ambos han aprendido a generalizar bien a nuevos datos y no han memorizado los datos de entrenamiento.

La razon por la que las metricas son cercanas a 1.0 en ambas regiones puede deberse a que los modelos son muy buenos en la tarea de clasificacion de los hongos. Y tambien puede ser que los datos en si mismos sean faciles de clasificar, es decir, que las caracteristicas de los hongos comestibles y venenosos sean muy distintas y faciles de distinguir.